# Penguin Analysis with jupyter-cc

This notebook demonstrates using `%cc` magic to work with Claude Code inside Jupyter.
Data is loaded with regular Python — `%cc` is used for analysis and visualization.

In [ ]:
%load_ext jupyter_cc_magic

## Load Data

In [ ]:
import seaborn as sns

df = sns.load_dataset("penguins").dropna()
df.head()

## Visualize

Ask Claude to create a scatter plot from the loaded data.

In [ ]:
%cc Create a scatter plot of bill_length_mm vs bill_depth_mm, colored by species. Use matplotlib.

## Summary Statistics

Start a fresh conversation to calculate summary stats.

In [ ]:
%cc_new Calculate summary statistics for body_mass_g grouped by species and sex

## In-place Editing

Use `%cc_cur` to replace the prompt cell with generated code.

In [ ]:
%cc_cur Create a function that takes a species name and returns a dict of mean measurements from df